# Setup

In [359]:
%pip install OpenAI

Note: you may need to restart the kernel to use updated packages.


In [360]:

from IPython.display import display, Markdown
import os
import ipywidgets as widgets

### Pretty printing help

In [361]:
import json

def show_json(obj):
    display(json.loads(obj.model_dump_json()))

### Load environment variables

In [362]:
from dotenv import load_dotenv
load_dotenv()  # Load the environment variables from .env file

True

## Choose the Example

The following will let the user switch to a different example to run through the pipeline.

In [533]:
def on_button_clicked(b):
  # This function will be called when the button is clicked
  global choice, job_description_file, company_file, destination_directory, html_destination, pdf_destination, markdown_destination, word_destination
  choice = choice_widget.value
  print("You selected: " + choice)
  # source files
  job_description_file = "examples/" + choice + "/job-description.md"
  company_file = "examples/" + choice + "/about-company.md"

  # destination files
  destination_directory = "examples/" + choice + "/"
  html_destination = destination_directory + 'playbook.html'
  pdf_destination = destination_directory + 'playbook.pdf'
  markdown_destination = destination_directory + 'playbook.md'
  word_destination = destination_directory + 'playbook.docx'


# Which example to run
choice_widget = widgets.Dropdown(
  options=['medical-assistant', 'infra-engineer', 'customer-success-manager', 'real-estate-litigator'],
  value='medical-assistant',
  description='Choose an example:'
)
choice = choice_widget.value

# source files
job_description_file = "examples/" + choice + "/job-description.md"
company_file = "examples/" + choice + "/about-company.md"

# destination files
destination_directory = "examples/" + choice + "/"
html_destination = destination_directory + 'playbook.html'
pdf_destination = destination_directory + 'playbook.pdf'
markdown_destination = destination_directory + 'playbook.md'
word_destination = destination_directory + 'playbook.docx'

button = widgets.Button(description="Confirm")


button.on_click(on_button_clicked)

display(choice_widget, button)



Dropdown(description='Choose an example:', options=('medical-assistant', 'infra-engineer', 'customer-success-m…

Button(description='Confirm', style=ButtonStyle())

You selected: infra-engineer
You selected: customer-success-manager


### Job Description

Read the job description into a variable.

In [535]:
job_description_file = "examples/" + choice + "/job-description.md"
with open(job_description_file, "r") as file:
    job_description = file.read()

print(job_description)


# Job Description

## Job Title
Customer Success Manager

## Job Description
Frontier Shield is seeking a passionate Customer Success Manager to ensure our customers get the maximum value from our cybersecurity solutions. You will be the key point of contact for assigned customers focused on driving adoption, retention, and satisfaction.

## Responsibilities
- Onboard new customers, conduct training, and provide support to guarantee a smooth implementation.
- Build close relationships with customers and serve as their trusted advisor for our products and services.
- Proactively communicate with customers to ensure they are leveraging our solutions effectively and reaching their security goals.
- Identify expansion opportunities within accounts and work cross-functionally to drive renewal and growth.
- Monitor customer health metrics like usage, user adoption, and support requests to anticipate needs.
- Manage escalated issues and troubleshoot problems quickly to maintain high customer

### Company Description

Read the company description into a variable.

In [510]:

company_file = "examples/" + choice + "/about-company.md"

with open(company_file, "r") as file:
    company_description = file.read()

print(company_description)

## Maxwell Fairview: Mission, Vision, and Values

### Mission
Maxwell Fairview's mission is centered around working collaboratively to address complex challenges across various legal domains, such as litigation, property law, corporate matters, and intellectual property, on an international stage. The firm is committed to delivering top-notch service and ensuring the success of its clients, consistently striving to surpass expectations.

### Vision
The vision of Maxwell Fairview is to make a meaningful difference for its clients and the communities it serves. This is achieved through a combination of deep industry insight and experience, offering innovative solutions for intricate legal issues, cultivating and upholding trust, and nurturing a culture rooted in respect, integrity, and inclusivity.

### Values
Maxwell Fairview's values are embodied in their dedication to:
- **Unparalleled Service:** Emphasizing service excellence and client success, always aiming to exceed in every facet

# Develop top-level list of competencies

## Setup OpenAI

In [511]:
from openai import OpenAI
client = OpenAI()
llm = "gpt-4-1106-preview"

## Get Draft List of Competencies

In [512]:
prompt = f"""
You are a Talent Acquisition and Development Specialist. Your task is to do a job analysis and identify the competencies for a hiring team to 
assess in a candidate. Lower level jobs should have less competencies and higher level jobs should have more, but you should cover all the
requirements in the job desccription. You should also include competencies that are aligned with the company values.
If a competency is aligned with a company value, you should say so.
Do not include acronyms or terminology that would not be understood by an industry professional not from the company.
Refer to the actual company name rather than 'organization' or 'company'.
Don't use the word 'technological' instead use 'technology' or 'technical'.

The job description is as follows:
{job_description}

Refer to the company description to ensure that the competencies are aligned with the company values as follows:
{company_description}

Return the key competencies for the job description as a JSON object. Use the following format:

{{
  "role": "Role Name",
  "company": "Company Name",
  "thinking": "Your thoughts about how to tackle this problem",
  "competencies": [
    {{
      "name": "Competency 1",
      "description": "Description of Competency 1",
    }},
    {{
      "name": "Competency 2",
      "description": "Description of Competency 2",
   }},
    {{
      "name": "Competency 3",
      "description": "Description of Competency 3",
    }}
  ]
}}

"""

stream = client.chat.completions.create(
    model=llm,
    max_tokens=4096,
    response_format={"type": "json_object"},
    messages=[{"role": "user", "content": prompt}],
    stream=True,
)


output = ""

for chunk in stream:
    if chunk.choices[0].delta.content is not None:
        print(chunk.choices[0].delta.content, end="")
        output += chunk.choices[0].delta.content

competency_data = json.loads(output)
role = competency_data["role"]
company = competency_data["company"]

{
  "role": "Real Estate Litigation Attorney",
  "company": "Maxwell Fairview",
  "thinking": "To determine the appropriate competencies for the Real Estate Litigation Attorney role at Maxwell Fairview, I reviewed the job description and mapped the required skills and experience to the company's mission, vision, and values. Given the nature of the role, which deals with complex legal challenges in construction and real estate litigation, I focused on identifying core competencies related to legal expertise, interpersonal skills, and personal attributes that reflect the company's commitment to service, experience, trust, and inclusion.",
  "competencies": [
    {
      "name": "Legal Expertise in Construction and Real Estate Litigation",
      "description": "Possessing a strong foundation in commercial and real estate litigation with the ability to manage a varied caseload involving complex issues. At least three to six years of relevant experience and California Bar admission are crit

## Review and improve the list of competencies

In [513]:
prompt = f"""
You are a Talent Acquisition and Development Specialist. Your task is to do a job analysis and review the key competencies 
for a hiring team to look for in a candidate. Evaluate the provided list of competencies and improve them if necessary.
You will examine the provided competency framework with an eye towards comprehensiveness, relevance, and alignment with industry best practices. 
Your evaluation will focus on ensuring that the framework effectively covers all necessary areas for the {role} at {company}.
Junior or low-level jobs should have fewer competencies (e.g., 6) and more senior jobs should have more (e.g., 10).
Do not include acronyms or terminology that would not be understood by an industry professional not from the company.
Refer to the actual company name wherever possible rather than use a generic word such as 'organization' or 'company'.
Don't use the word 'technological' instead use 'technology' or 'technical'.

If you think the list of competencies is good, you can leave it as is.
If you think the list of competencies is bad, you can delete it and write a new one.
If you think the list of competencies is good but needs improvement, you can edit it.
If you think the list of competencies is good but needs more competencies, you can add more.
If you think the list of competencies is good but needs less competencies, you can delete some.
If you think the list of competencies is good but needs to be re-ordered, you can re-order them.
If you think the list of competencies is good but needs to be re-worded, you can re-word them.
If a description is aligned with a company value, you should say so.

The job description is as follows:
{job_description}

The current list of competencies is as follows:
{output}

Refer to the company description to ensure that the competencies are aligned with the company values as follows:
{company_description}

Return the improved key competency list as a JSON object. Use the following format:

{{
  "role": "Role Name",
  "company": "Company Name",
  "competencies": [
    {{
      "name": "Competency 1",
      "description": "Description of Competency 1",
      "improvements": "Improvements made (if any)"
    }},
    {{
      "name": "Competency 2",
      "description": "Description of Competency 2",
      "improvements": "Improvements made (if any)"
   }},
    {{
      "name": "Competency 3",
      "description": "Description of Competency 3",
      "improvements": "Improvements made (if any)"
    }}
  ]
}}

"""

stream = client.chat.completions.create(
    model=llm,
    max_tokens=4096,
    response_format={"type": "json_object"},
    messages=[{"role": "user", "content": prompt}],
    stream=True,
)

output = ""

for chunk in stream:
    if chunk.choices[0].delta.content is not None:
        print(chunk.choices[0].delta.content, end="")
        output += chunk.choices[0].delta.content

competency_data = json.loads(output)


{
  "role": "Real Estate Litigation Attorney",
  "company": "Maxwell Fairview",
  "competencies": [
    {
      "name": "In-depth Legal Expertise in Construction and Real Estate Litigation",
      "description": "Demonstrates a robust understanding of commercial and real estate litigation, with the ability to handle a multifaceted caseload involving sophisticated legal issues.",
      "improvements": "Clarified the expertise area by specifying 'in-depth knowledge' to emphasize the depth of experience expected."
    },
    {
      "name": "Analytical and Strategic Problem-Solving",
      "description": "Exhibits a meticulous approach to analyzing legal issues, devising strategic solutions, and translating complex laws into solid legal arguments.",
      "improvements": "Added 'strategic' to highlight the need for long-term thinking in developing legal solutions."
    },
    {
      "name": "Exceptional Communication and Persuasion Skills",
      "description": "Possesses top-tier commun

## Create Markdown for Competency Framework Introduction

In [527]:
markdown_competencies_intro = "\n"
counter = 1
for item in competency_data["competencies"]:
    markdown_competencies_intro += f"""
{counter}. **{item["name"]}**: {item["description"]}

"""
    counter += 1

markdown_competencies_intro = f"""

### Competency Framework

The following competencies have been identified for the {role} role at {company}:

{markdown_competencies_intro}

"""

display(Markdown(markdown_competencies_intro))



### Competency Framework

The following competencies have been identified for the Real Estate Litigation Attorney role at Maxwell Fairview:



1. **In-depth Legal Expertise in Construction and Real Estate Litigation**: Demonstrates a robust understanding of commercial and real estate litigation, with the ability to handle a multifaceted caseload involving sophisticated legal issues.


2. **Analytical and Strategic Problem-Solving**: Exhibits a meticulous approach to analyzing legal issues, devising strategic solutions, and translating complex laws into solid legal arguments.


3. **Exceptional Communication and Persuasion Skills**: Possesses top-tier communication abilities for effective dialogue with clients and peers, and the skill to persuade in both written and oral advocacy.


4. **Client-Centric Service Focus**: Upholds a client-first approach, preemptively recognizing client requirements and aiming for excellence in legal representation, embodying Maxwell Fairview’s unparalleled service value.


5. **Effective Collaboration and Teamwork**: Thrives in a collaborative environment, contributing to and benefiting from the combined wisdom of the team, in line with Maxwell Fairview’s rich experience value.


6. **Adaptability in a Dynamic Environment**: Displays flexibility to navigate a fast-evolving legal landscape and the ability to manage concurrent priorities with efficacy.


7. **Advocacy for Diversity and Inclusion**: Actively promotes and integrates inclusive practices within the legal framework, mirroring Maxwell Fairview’s dedication to respectful inclusion.


8. **Ethical Judgement and Professional Integrity**: Consistently exhibits ethical behavior and integrity in professional dealings, foundational to earning and preserving unshakeable trust with clients and peers.


9. **Technological Proficiency**: Leverages advanced technology tools for legal research, case management, and client communication, staying ahead in a tech-driven legal sector.


10. **Commitment to Continuing Legal Education**: Continually seeks knowledge growth and professional development to stay updated with legal trends and regulations, reflecting the firm’s commitment to rich experience and unparalleled service.





## Evaluation scorecard markdown

In [515]:
markdown_evaluation_scorecard = f"""

## Interview Evaluation for {role}

| Competency | N/A | 1 | 2 | 3 | 4 | 5 |
| --- | --- | --- | --- | --- | --- | --- |
"""

counter = 1
for item in competency_data["competencies"]:
    markdown_evaluation_scorecard += f"""| {counter}. {item["name"]} | | | | | | |
"""
    counter += 1

spacer = " "

markdown_evaluation_scorecard += f"""| &nbsp; | | | | | | |
| &nbsp; | | | | | | |
| &nbsp; | | | | | | |
| &nbsp; | | | | | | |
| &nbsp; | **N/A** | **1** | **2** | **3** | **4** | **5** |
| **Totals** | {spacer} | {spacer} | {spacer} | {spacer} | {spacer} | {spacer} |

"""

display(Markdown(markdown_evaluation_scorecard))



## Interview Evaluation for Real Estate Litigation Attorney

| Competency | N/A | 1 | 2 | 3 | 4 | 5 |
| --- | --- | --- | --- | --- | --- | --- |
| 1. In-depth Legal Expertise in Construction and Real Estate Litigation | | | | | | |
| 2. Analytical and Strategic Problem-Solving | | | | | | |
| 3. Exceptional Communication and Persuasion Skills | | | | | | |
| 4. Client-Centric Service Focus | | | | | | |
| 5. Effective Collaboration and Teamwork | | | | | | |
| 6. Adaptability in a Dynamic Environment | | | | | | |
| 7. Advocacy for Diversity and Inclusion | | | | | | |
| 8. Ethical Judgement and Professional Integrity | | | | | | |
| 9. Technological Proficiency | | | | | | |
| 10. Commitment to Continuing Legal Education | | | | | | |
| &nbsp; | | | | | | |
| &nbsp; | | | | | | |
| &nbsp; | | | | | | |
| &nbsp; | | | | | | |
| &nbsp; | **N/A** | **1** | **2** | **3** | **4** | **5** |
| **Totals** |   |   |   |   |   |   |



# Detailed Competencies

## Function for analyzing a competency in depth

In [516]:
def analyze_competency(competency, competency_description):
    prompt = f"""
You are an Interviewing and Assessment Expert. Your task is to do an analysis of the competency "{competency}" for a job to give to a hiring team. 

You should generate a great interview question to assess "{competency}". It should be the BEST question you could ask. 
The question should include 1-4 short follow-up questions embedded in it. 

Also produce what to look for in the answer to the lead question when evaluating the candidate's answer.

You will then generate 3 or 4 subcompetencies for the "{competency}" competency. Each subcompetency should have a name and a 
question to ask the candidate.

You will then generate a scenario for "{competency}" to ask the candidate to respond to.

You will then generate a set of VERY concise evaluation rubrics of less than 20 words to use when evaluating the candidate's answer 
to the lead question and the scenario.
Assume we are using a 5 point scale where 1 is the lowest rating and 5 is the highest rating.
Assume we know we are rating the candidate and don't refer to the candidate.

The competency description is as follows:
{competency_description}

The job description is as follows:
{job_description}

Refer to the company description to ensure that the competencies are aligned with the company values as follows:
{company_description}

Return the competency analysis list as only a JSON object with no other commentary. Use the following format:

{{"competency": [{{
  "name": "Competency Name",
  "description": "Competency Description",
  "lead_question": "Lead Interview Question",
  "lead_look_for": "What to look for and evaluate in the answer to the lead question. Address the reader.",
  "subcompetencies": [
    {{
      "name": "Subcompetency 1",
      "question": "Interview question for Subcompetency"
    }},
    {{
      "name": "Subcompetency 2",
      "question": "Interview question for Subcompetency"
    }},
    {{
      "name": "Subcompetency 3",
      "question": "Interview question for Subcompetency"
    }}
  ],
  "scenario": "Scenario to ask the candidate to respond to",
  "scenario_look_for": "What to look for or evaluate in the answer to the scenario. Address the reader.",
  "rating_1": "What a level 1 rating for this competency looks like. Be very concise.",
  "rating_2: "What a level 2 rating for this competency looks like. Be very concise.",
  "rating_3": "What a level 3 rating for this competency looks like. Be very concise.",
  "rating_4": "What a level 4 rating for this competency looks like. Be very concise.",
  "rating_5": "What a level 5 rating for this competency looks like. Be very concise."
}}"""

    stream = client.chat.completions.create(
        model=llm,
        max_tokens=4096,
        response_format={"type": "json_object"},
        messages=[{"role": "user", "content": prompt}],
        stream=True,
    )

    output = ""

    for chunk in stream:
        if chunk.choices[0].delta.content is not None:
            print(chunk.choices[0].delta.content, end="")
            output += chunk.choices[0].delta.content

    return json.loads(output)


## Generate detailed competencies

In [517]:
competency_details = {"competencies":[]}

for i, competency in enumerate(competency_data["competencies"]):
    if i == 11:
        break
    competency_deep_dive = analyze_competency(competency["name"], competency["description"])
    competency_details["competencies"].append(competency_deep_dive)

{
  "competency": [{
    "name": "In-depth Legal Expertise in Construction and Real Estate Litigation",
    "description": "Demonstrates a robust understanding of commercial and real estate litigation, with the ability to handle a multifaceted caseload involving sophisticated legal issues.",
    "lead_question": "Describe a complex construction or real estate litigation case you have handled. What were the key legal issues, and how did you address them? How did you ensure a positive outcome for your client? Were there any unique challenges, and if so, how did you overcome them?",
    "lead_look_for": "Look for detailed understanding of legal concepts specific to real estate and construction law, strategic thinking, problem-solving abilities, and successful outcomes.",
    "subcompetencies": [
      {
        "name": "Understanding of Construction Law",
        "question": "What are the most common types of disputes in construction law you've encountered, and how have you resolved them?

In [518]:
display_json = json.dumps(competency_details, indent=2)
print(display_json)

{
  "competencies": [
    {
      "competency": [
        {
          "name": "In-depth Legal Expertise in Construction and Real Estate Litigation",
          "description": "Demonstrates a robust understanding of commercial and real estate litigation, with the ability to handle a multifaceted caseload involving sophisticated legal issues.",
          "lead_question": "Describe a complex construction or real estate litigation case you have handled. What were the key legal issues, and how did you address them? How did you ensure a positive outcome for your client? Were there any unique challenges, and if so, how did you overcome them?",
          "lead_look_for": "Look for detailed understanding of legal concepts specific to real estate and construction law, strategic thinking, problem-solving abilities, and successful outcomes.",
          "subcompetencies": [
            {
              "name": "Understanding of Construction Law",
              "question": "What are the most common ty

## Competency detail sections markdown

In [523]:
counter = 1
markdown_competencies = "\n"

for competency_group in competency_details["competencies"]:
  for competency_item in competency_group["competency"]:

    markdown_competencies += "## Competency " + str(counter) + ". **" 
    markdown_competencies += competency_item["name"] + "**\n\n"
    markdown_competencies += competency_item["description"] + "\n\n"
    markdown_competencies += "### Lead Question\n\n"
    markdown_competencies += "> **" + competency_item["lead_question"] + "** \n\n"
    markdown_competencies += "_" + competency_item["lead_look_for"] + "_\n\n"

    markdown_competencies += "\n### Scenario\n\n"
    markdown_competencies += "> " + competency_item["scenario"] + "\n\n"
    markdown_competencies += "_" + competency_item["scenario_look_for"] + "_\n\n"

    markdown_competencies += "### Diving Deeper\n\n"
    for subcompetency in competency_item["subcompetencies"]:
      markdown_competencies += "* **" + subcompetency["name"] + "**: \"" + subcompetency["question"] + "\"\n"


    markdown_competencies += "\n### Evaluation\n\n"
    markdown_competencies += "| Rating 1 | Rating 2 | Rating 3 | Rating 4 | Rating 5 |\n"
    markdown_competencies += "| -------- | -------- | -------- | -------- | -------- |\n"
    markdown_competencies += "|" + competency_item["rating_1"] + "|" + competency_item["rating_2"] + "|" + competency_item["rating_3"] + "|" + competency_item["rating_4"] + "|" + competency_item["rating_5"] + "|\n\n"
    
    markdown_competencies += "\n### Notes\n\n&nbsp;\n\n"

    counter += 1




# Final markdown

In [524]:
from datetime import datetime

# Get the current date and time
now = datetime.now()

# Format the date and time in a nice readable format
formatted_now = now.strftime("%B %d, %Y %-I:%M %p")

print(formatted_now)

January 09, 2024 8:55 AM


In [525]:
# Build final markdown

final_markdown = f"""--- 
title: Interview Guide for {role}
subtitle: {company}
author: Created by [Deliverables.ai](https://github.com/androb/deliverables)
date: {formatted_now}
--- 

# Interview Guide for {role}

## About this Guide

Welcome to this strategic guide crafted to streamline and enhance the interview process for the position of {role}. 
This document serves as a comprehensive resource for hiring teams, ensuring a structured, efficient, and effective evaluation of candidates. 
The role of {role} is crucial to {company}, bridging vital functions and contributing significantly to our success. Our aim is to identify 
a candidate who not only possesses the necessary technical skills and experience but also aligns with our company's values and culture. 

The playbook outlines a clear structure for the interview, including a detailed assessment of key competencies, practical assignments, and behavioral insights. 
Through this approach, we aspire to understand each candidate's potential and ensure the best fit for both the role and {company}.

This interview guide was created by [Deliverables AI](https://www.deliverables.ai) and is based on the job description and company description provided.
It should be used in conjunction with an interviewer's expertise and knowledge of the domain to assess the candidate's suitability for the role. 
Research shows that structured interviews are significantly more effective and less biased at identifying successful future employees and we welcome any 
feedback you have to [feedback@deliverables.ai](mailto:feedback@deliverables.ai).

{markdown_competencies_intro}

{company_description}

{job_description}


{markdown_competencies}

{markdown_evaluation_scorecard}
"""

display(Markdown(final_markdown))

# Write playbook to a text file
with open("test.md", 'w') as file:
    file.write(final_markdown)

--- 
title: Interview Guide for Real Estate Litigation Attorney
subtitle: Maxwell Fairview
author: Created by [Deliverables.ai](https://github.com/androb/deliverables)
date: January 09, 2024 8:55 AM
--- 

# Interview Guide for Real Estate Litigation Attorney

## About this Guide

Welcome to this strategic guide crafted to streamline and enhance the interview process for the position of Real Estate Litigation Attorney. 
This document serves as a comprehensive resource for hiring teams, ensuring a structured, efficient, and effective evaluation of candidates. 
The role of Real Estate Litigation Attorney is crucial to Maxwell Fairview, bridging vital functions and contributing significantly to our success. Our aim is to identify 
a candidate who not only possesses the necessary technical skills and experience but also aligns with our company's values and culture. 

The playbook outlines a clear structure for the interview, including a detailed assessment of key competencies, practical assignments, and behavioral insights. 
Through this approach, we aspire to understand each candidate's potential and ensure the best fit for both the role and Maxwell Fairview.

This interview guide was created by [Deliverables AI](https://www.deliverables.ai) and is based on the job description and company description provided.
It should be used in conjunction with an interviewer's expertise and knowledge of the domain to assess the candidate's suitability for the role. 
Research shows that structured interviews are significantly more effective and less biased at identifying successful future employees and we welcome any 
feedback you have to [feedback@deliverables.ai](mailto:feedback@deliverables.ai).



### Competency Framework

The following competencies have been identified for the Real Estate Litigation Attorney role at Maxwell Fairview:



1. **In-depth Legal Expertise in Construction and Real Estate Litigation:** Demonstrates a robust understanding of commercial and real estate litigation, with the ability to handle a multifaceted caseload involving sophisticated legal issues.


2. **Analytical and Strategic Problem-Solving:** Exhibits a meticulous approach to analyzing legal issues, devising strategic solutions, and translating complex laws into solid legal arguments.


3. **Exceptional Communication and Persuasion Skills:** Possesses top-tier communication abilities for effective dialogue with clients and peers, and the skill to persuade in both written and oral advocacy.


4. **Client-Centric Service Focus:** Upholds a client-first approach, preemptively recognizing client requirements and aiming for excellence in legal representation, embodying Maxwell Fairview’s unparalleled service value.


5. **Effective Collaboration and Teamwork:** Thrives in a collaborative environment, contributing to and benefiting from the combined wisdom of the team, in line with Maxwell Fairview’s rich experience value.


6. **Adaptability in a Dynamic Environment:** Displays flexibility to navigate a fast-evolving legal landscape and the ability to manage concurrent priorities with efficacy.


7. **Advocacy for Diversity and Inclusion:** Actively promotes and integrates inclusive practices within the legal framework, mirroring Maxwell Fairview’s dedication to respectful inclusion.


8. **Ethical Judgement and Professional Integrity:** Consistently exhibits ethical behavior and integrity in professional dealings, foundational to earning and preserving unshakeable trust with clients and peers.


9. **Technological Proficiency:** Leverages advanced technology tools for legal research, case management, and client communication, staying ahead in a tech-driven legal sector.


10. **Commitment to Continuing Legal Education:** Continually seeks knowledge growth and professional development to stay updated with legal trends and regulations, reflecting the firm’s commitment to rich experience and unparalleled service.





## Maxwell Fairview: Mission, Vision, and Values

### Mission
Maxwell Fairview's mission is centered around working collaboratively to address complex challenges across various legal domains, such as litigation, property law, corporate matters, and intellectual property, on an international stage. The firm is committed to delivering top-notch service and ensuring the success of its clients, consistently striving to surpass expectations.

### Vision
The vision of Maxwell Fairview is to make a meaningful difference for its clients and the communities it serves. This is achieved through a combination of deep industry insight and experience, offering innovative solutions for intricate legal issues, cultivating and upholding trust, and nurturing a culture rooted in respect, integrity, and inclusivity.

### Values
Maxwell Fairview's values are embodied in their dedication to:
- **Unparalleled Service:** Emphasizing service excellence and client success, always aiming to exceed in every facet of their work.
- **Rich Experience:** Harnessing the collective knowledge and expertise of their team to provide novel solutions to legal challenges.
- **Unshakeable Trust:** Establishing and maintaining trust through consistent actions and behaviors, fostering robust, lasting relationships with clients and colleagues.
- **Respectful Inclusion:** Promoting a culture where respect and integrity are paramount, valuing and harnessing the diversity of people, ideas, and experiences for effective thinking and action.

Maxwell Fairview also prioritizes diversity, equity, and inclusion, sustainability, social impact, and the utilization of technology for innovation and enhanced client services.

These principles signify Maxwell Fairview’s commitment to providing superior legal services while upholding strong ethical standards and positively influencing both society and the environment.

## Real Estate Litigation Attorney

**Position:** Real Estate Litigation Attorney  
**Location:** San Francisco, CA  
**Practice Area:** Construction & Real Estate  
**Open Date:** Friday, December 15, 2023  
**Job Function:** Legal  

### Job Description:
At Maxwell Fairview, our priority is to attract, retain, and promote talented individuals from a wide range of racial, ethnic, social, economic, religious, and personal backgrounds, genders, and sexual orientations. Therefore, we encourage talented individuals with diverse backgrounds and experiences to apply.

More than a third of Maxwell Fairview’s lawyers are litigators. We have decades of experience representing clients at every level of the trial and appellate courts in multiple jurisdictions, in international forums, and mediations, arbitrations, and other methods of alternative dispute resolution. 

A career at Maxwell Fairview is the opportunity to do work that matters. It’s a chance to use your knowledge to shape what’s ahead, to innovate, and to learn at a firm that taps into the power of collaboration and collective thinking. 

In accordance with applicable Federal and State laws, the anticipated annual salary range for this position is as follows:

- San Francisco, CA: $160,000-$175,000

In addition to a standard benefits package, this role may be eligible for additional contingent compensation based on an array of factors, including but not limited to: work performance, geographic location, work experience, education, and qualifications. Because such contingent compensation is not yet calculable or may be zero ($0) in some circumstances, the above-listed salary range is Maxwell Fairview’s good faith estimate of the annual salary it reasonably expects to pay for the position at the time of this posting.

### Job Requirements:
Our Construction and Real Estate Litigation practice group is seeking to hire a department attorney to join our San Francisco, CA office. The successful candidate will work with our team on California landlord-tenant matters.  

Qualified candidates will have at least three to six years of commercial and real estate litigation experience. Ability to work in a fast-paced client-centric environment with a varied caseload involving complex issues is essential. California Bar admission is required.

Principals Only.

Maxwell Fairview LLP is an Equal Opportunity / Affirmative Action Employer.
Disability / Female / Gender Identity / Minority / Sexual Orientation / Veteran. 

Pursuant to the San Francisco Fair Chance Ordinance and the Los Angeles Fair Chance Initiative, we will consider for employment qualified applicants with arrest and conviction records. 

To comply with Federal law, Maxwell Fairview participates in E-Verify. All newly-hired employees are verified through this electronic system established by the Department of Homeland Security (DHS) and the Social Security Administration (SSA) to confirm their identity and employment eligibility. Please refer to the Notice of E-Verify Participation and the Right to Work posters on the Maxwell Fairview Careers page for more information.




## Competency 1. **In-depth Legal Expertise in Construction and Real Estate Litigation**

Demonstrates a robust understanding of commercial and real estate litigation, with the ability to handle a multifaceted caseload involving sophisticated legal issues.

### Lead Question

> **Describe a complex construction or real estate litigation case you have handled. What were the key legal issues, and how did you address them? How did you ensure a positive outcome for your client? Were there any unique challenges, and if so, how did you overcome them?** 

_Look for detailed understanding of legal concepts specific to real estate and construction law, strategic thinking, problem-solving abilities, and successful outcomes._


### Scenario

> A client is facing a high-stakes dispute regarding a large construction project, with allegations of contract breach by the contractor. Multiple subcontractors are also involved. How would you navigate this litigation while maintaining the client's interests?

_Assess strategy formulation, multi-party dynamics management, client focus, and risk assessment._

### Diving Deeper

* **Understanding of Construction Law**: "What are the most common types of disputes in construction law you've encountered, and how have you resolved them?"
* **Real Estate Acumen**: "Can you explain the intricacies involved in a real estate transaction gone to litigation?"
* **Litigation Strategies**: "How do you develop and adapt your litigation strategy to accommodate different judges, venues, or opposing counsel in real estate cases?"
* **Client Management in Litigation**: "How do you align client expectations with the realities of litigation in the context of construction and real estate law?"

### Evaluation

| Rating 1 | Rating 2 | Rating 3 | Rating 4 | Rating 5 |
| -------- | -------- | -------- | -------- | -------- |
|Minimal knowledge or ability, unclear or incorrect answers.|Basic understanding, lacks depth or strategic thinking.|Competent with room for improvement, shows potential.|Strong grasp of content, strategically and tactically adept.|Expert-level insight, demonstrates superior strategic acumen.|


### Notes

&nbsp;

## Competency 2. **Analytical and Strategic Problem-Solving**

Exhibits a meticulous approach to analyzing legal issues, devising strategic solutions, and translating complex laws into solid legal arguments.

### Lead Question

> **Describe a time when you faced a complex legal issue. How did you analyze the situation, develop a strategy, and argue your case? Could you walk us through your thought process and the outcome?** 

_Listen for a clear, logical approach, strategic thinking, and the ability to simplify complex concepts and craft persuasive legal arguments._


### Scenario

> A client is facing a potential lawsuit over a disputed property line. As new constructions are in development, they could infringe on the disputed space. How would you approach the analysis and resolution of this case?

_Expect analytical breakdown of the issue, strategic resolution planning, incorporating client's interests, and clear communication skills._

### Diving Deeper

* **Critical Thinking**: "How do you differentiate between correlation and causation when analyzing case data?"
* **Decision Making**: "Can you tell us about a difficult legal decision you had to make and how you approached it?"
* **Legal Research and Interpretation**: "Explain how you stay current with changes in real estate law and apply them to your litigation strategies."
* **Client-Centric Approach**: "How do you ensure that your legal strategies align with your client's objectives and risk tolerance?"

### Evaluation

| Rating 1 | Rating 2 | Rating 3 | Rating 4 | Rating 5 |
| -------- | -------- | -------- | -------- | -------- |
|Unstructured thought; misses critical elements.|Basic analysis; simplistic strategy; lacks depth.|Adequate analysis and strategy with minor oversight.|Strong analysis; clear, strategic, client-aligned thinking.|Exceptional insight; strategic mastery; client-focused.|


### Notes

&nbsp;

## Competency 3. **Exceptional Communication and Persuasion Skills**

Possesses top-tier communication abilities for effective dialogue with clients and peers, and the skill to persuade in both written and oral advocacy.

### Lead Question

> **Can you tell us about a time you successfully persuaded a client or colleague to take a particular course of action that they were initially resistant to, and how did you manage to change their perspective?** 

_Assess the candidate's ability to identify the audience's needs, articulate benefits, tailor the message, handle objections, and achieve the desired outcome._


### Scenario

> Imagine a client is opposed to a settlement offer that you believe is in their best interest. How would you use your communication skills to persuade the client to consider the settlement?

_Evaluate how the candidate demonstrates understanding the client's position, builds rapport, communicates the offer's advantages, and ethically persuades._

### Diving Deeper

* **Active Listening**: "Describe a situation where active listening helped you understand and resolve a client's concern."
* **Clarity and Articulation**: "Provide an example of a complex legal concept you had to explain to a non-legal audience. How did you ensure they understood?"
* **Emotional Intelligence**: "Recall a time your emotional intelligence helped navigate and resolve a difficult situation in the workplace. What was your approach?"
* **Conflict Resolution**: "Tell me about a time you mediated a dispute between parties in a case. How did you persuade each party towards a resolution?"

### Evaluation

| Rating 1 | Rating 2 | Rating 3 | Rating 4 | Rating 5 |
| -------- | -------- | -------- | -------- | -------- |
|Inadequate understanding or articulation; no evidence of persuasion.|Limited persuasion; basic communication but lacking depth or efficacy.|Competent communication; some evidence of persuasion and understanding.|Strong, clear communication; good persuasion with some resistance overcome.|Exceptional persuasion and communication; overcomes resistance with ease.|


### Notes

&nbsp;

## Competency 4. **Client-Centric Service Focus**

Upholds a client-first approach, preemptively recognizing client requirements and aiming for excellence in legal representation, embodying Maxwell Fairview’s unparalleled service value.

### Lead Question

> **Can you describe a time when you had to go above and beyond to ensure client satisfaction while maintaining professional integrity, especially in a complex legal scenario? What was the situation, what actions did you take, and what was the outcome?** 

_Look for examples demonstrating understanding of client needs, taking initiative, providing exceptional service, and balancing professional integrity._


### Scenario

> Imagine you're representing a client who is facing a time-sensitive and complex litigation matter. The client has a limited understanding of the legal processes but insists on a quick resolution. How would you manage their expectations while ensuring a thorough and strategic legal approach?

_Assess candidates' ability to manage client expectations, provide clear explanations, and maintain a strategic approach._

### Diving Deeper

* **Understanding Client Needs**: "How do you approach understanding and prioritizing your clients' goals and expectations in a legal dispute?"
* **Responsiveness and Communication**: "Give an example of how you have effectively communicated complex legal information to a client to ensure their understanding."
* **Problem-Solving and Innovation**: "Tell me about a time when you had to find an innovative solution to a client's legal problem. What was innovative about your approach?"
* **Balancing Client Interests with Ethical Standards**: "How would you handle a situation where a client’s request could potentially conflict with ethical guidelines or legal standards?"

### Evaluation

| Rating 1 | Rating 2 | Rating 3 | Rating 4 | Rating 5 |
| -------- | -------- | -------- | -------- | -------- |
|Does not understand clients' needs; ignores professional integrity.|Limited recognition of client needs; struggles to maintain integrity.|Addresses client needs adequately; maintains basic professional integrity.|Actively identifies and fulfills client needs; good professional integrity.|Exemplary understanding and proactive service; impeccable integrity.|


### Notes

&nbsp;

## Competency 5. **Effective Collaboration and Teamwork**

Thrives in a collaborative environment, contributing to and benefiting from the combined wisdom of the team, in line with Maxwell Fairview’s rich experience value.

### Lead Question

> **Can you describe a time when you had to collaborate with a team to solve a complex litigation matter? What role did you play in fostering teamwork? How did you ensure all voices were heard? Were there any challenges, and how did you overcome them?** 

_Look for specific examples of collaborative work, the candidate's ability to facilitate team dynamics, promote inclusivity, and resolve conflicts effectively._


### Scenario

> Imagine your team is divided on the best litigation approach to take in a high-profile real estate case. How would you facilitate the decision-making process to reach a unified strategy?

_Evaluator should seek demonstration of conflict resolution, decision-making skills, ability to unite the team, and leadership qualities._

### Diving Deeper

* **Integrative Thinking**: "How do you merge diverse ideas from team members to arrive at a solution that addresses complex legal issues?"
* **Communication and Influence**: "How have you persuaded your team to adopt a new idea or change direction on a project?"
* **Conflict Resolution**: "Describe how you resolved a dispute within your team and what the outcome was."
* **Leadership and Mentoring**: "Tell me about a time you had to lead a legal team. How did you ensure everyone's development and success?"

### Evaluation

| Rating 1 | Rating 2 | Rating 3 | Rating 4 | Rating 5 |
| -------- | -------- | -------- | -------- | -------- |
|Displays inability to collaborate or resolve conflicts; passive.|Shows minimal effort to engage in teamwork; occasional conflicts.|Generally works well in teams but may struggle with complex conflicts.|Consistently collaborates well and can often resolve conflicts.|Excels at collaboration, seamlessly integrates ideas, and resolves conflicts effectively.|


### Notes

&nbsp;

## Competency 6. **Adaptability in a Dynamic Environment**

Displays flexibility to navigate a fast-evolving legal landscape and the ability to manage concurrent priorities with efficacy.

### Lead Question

> **Can you describe a situation where you had to adjust your legal strategy due to an unexpected change in legislation or the business environment? How did you approach this challenge, what was the outcome, and what did you learn from the experience?** 

_Look for evidence of quick thinking, a proactive approach, problem-solving skills, and a positive attitude towards change. Also, evaluate how the outcome reflects flexibility, learning, and strategic adjustment._


### Scenario

> Imagine you are representing a client in a real estate dispute when a new housing regulation that affects your case comes into effect. How would you reassess your case strategy and advise your client?

_Look for analytical skills, strategic thinking, the ability to communicate changes effectively to clients, and an understanding of the implications of new regulations._

### Diving Deeper

* **Quick Thinking and Responsiveness**: "Tell me about a time when you had to make a quick decision in a complex case. How did you ensure that the decision was well-informed?"
* **Prioritization and Time Management**: "When managing multiple cases with tight deadlines, how do you determine your priorities and delegate tasks?"
* **Learning Agility**: "Give an example of how you have had to quickly learn a new area of law or regulation. What approach did you take, and how did it impact your work?"
* **Embracing Change**: "Share an experience where you embraced a significant change within your practice area. How did you adapt and what was the result?"

### Evaluation

| Rating 1 | Rating 2 | Rating 3 | Rating 4 | Rating 5 |
| -------- | -------- | -------- | -------- | -------- |
|Shows resistance to change, inability to adapt quickly.|Struggles with change; adapts, but slowly or reactively.|Adapts to change adequately; shows some proactive strategy.|Adapts well, with proactive strategies and efficiency.|Excels in adaptability, turning changes into opportunities.|


### Notes

&nbsp;

## Competency 7. **Advocacy for Diversity and Inclusion**

Actively promotes and integrates inclusive practices within the legal framework, mirroring Maxwell Fairview’s dedication to respectful inclusion.

### Lead Question

> **Can you describe a situation where you actively advocated for diversity and inclusion in the workplace? How did you recognize a need for it, what did you do, what was the outcome, and how did you gauge its effectiveness?** 

_Seek demonstrations of understanding and commitment to D&I. Look for tangible actions taken, effective outcomes, and methods for assessing impact._


### Scenario

> Imagine you've joined a team working on a major case, but you notice a lack of diverse perspectives that could potentially enhance the outcome. How would you approach this situation, and what steps might you take to advocate for the inclusion of more diverse viewpoints?

_Expect strategies to enhance D&I within teams. Evaluate communication, inclusivity in problem-solving, and proactive approach._

### Diving Deeper

* **Understanding of D&I Importance**: "Why do you believe diversity and inclusion are important in the legal sector, especially within litigation?"
* **Inclusive Leadership**: "How do you foster an inclusive environment for your team and address conflicts that arise from diversity?"
* **D&I Policies Implementation**: "Describe how you have contributed to the development or implementation of Diversity and Inclusion policies in a previous role."
* **Legal Advocacy for D&I**: "Can you give an example of how you have leveraged your legal expertise to advocate for social justice or influence diversity and inclusion positively?"

### Evaluation

| Rating 1 | Rating 2 | Rating 3 | Rating 4 | Rating 5 |
| -------- | -------- | -------- | -------- | -------- |
|Minimal acknowledgement or understanding; passive attitudes toward D&I.|Recognizes importance of D&I; lacking active engagement or examples.|Demonstrates understanding; some action taken, limited effectiveness.|Articulates and acts upon D&I principles; provides robust examples; measures impact.|Champions D&I; substantial impact; sets benchmarks; leads initiatives with measured success.|


### Notes

&nbsp;

## Competency 8. **Ethical Judgement and Professional Integrity**

Consistently exhibits ethical behavior and integrity in professional dealings, foundational to earning and preserving unshakeable trust with clients and peers.

### Lead Question

> **Describe a time when you faced an ethical dilemma at work. How did you address it, and what were the outcomes? Did you consult anyone in the decision-making process, and how did you ensure your actions aligned with professional integrity?** 

_Evaluate the ethical complexity, recognition of dilemma, process for resolution, consultation with relevant parties, and alignment with integrity._


### Scenario

> Imagine you have discovered evidence that could strongly benefit your client's case, but you have reason to believe this evidence was obtained unethically. How would you proceed?

_Assess for ethical decision-making, consideration of legal standards, impact assessment, and professional integrity._

### Diving Deeper

* **Understanding Ethical Standards**: "Can you provide an example of how you stay informed about updates on ethical standards in the legal profession?"
* **Balance Professional Duty and Personal Values**: "How do you handle situations where your personal values may conflict with professional responsibilities?"
* **Maintaining Confidentiality**: "What methods do you use to ensure that sensitive client information remains confidential?"
* **Accountability for Ethics Violations**: "Tell me about a time when you witnessed a colleague violating professional ethics. What actions did you take?"

### Evaluation

| Rating 1 | Rating 2 | Rating 3 | Rating 4 | Rating 5 |
| -------- | -------- | -------- | -------- | -------- |
|Displays poor recognition; unethical choices made.|Shows basic recognition; minor ethical actions taken.|Adequate recognition; follows ethics but with hesitancy.|Strong recognition and action; consults peers or standards.|Exceptional action; ethical leader, transparent process.|


### Notes

&nbsp;

## Competency 9. **Technological Proficiency**

Leverages advanced technology tools for legal research, case management, and client communication, staying ahead in a tech-driven legal sector.

### Lead Question

> **Can you describe a time where you utilized a specific technology to optimize a complex legal research task or case management process? How did it improve efficiency, accuracy, or client communication? Were there any challenges you faced, and how did you overcome them?** 

_Look for specific technology tools mentioned and how they were applied effectively to a legal context. Evaluate the candidate’s problem-solving skills in overcoming challenges with technology._


### Scenario

> Imagine you have been given a large set of legal documents to analyze for potential litigation. The documents are in various formats and contain complex data. Describe how you would leverage technology to efficiently manage, review, and extract relevant information from these documents.

_Look for a structured approach to data management, knowledge of relevant document review tools, attention to efficiency, accuracy, and ability to extract relevant information._

### Diving Deeper

* **Legal Research Technology**: "What legal research platforms are you proficient in, and how do you ensure you are using these tools effectively?"
* **Case Management Software**: "Which case management software solutions are you familiar with, and what features do you find most beneficial for managing cases?"
* **Client Communication Platforms**: "In what ways have you utilized technology to improve client communication, and can you provide an example of how you have maintained confidentiality and compliance?"
* **Adaptability to New Technologies**: "Tell me about a time when you had to quickly adapt to a new technology or software in your legal practice. How did you approach learning it?"

### Evaluation

| Rating 1 | Rating 2 | Rating 3 | Rating 4 | Rating 5 |
| -------- | -------- | -------- | -------- | -------- |
|Minimal or no relevant technology use mentioned.|Basic technology use mentioned, little to no impact described.|Competent use of technology with average impact and efficiency noted.|Advanced technology use with significant impact and challenges addressed.|Expert technology application with high impact, major efficiency gains, and innovative problem-solving.|


### Notes

&nbsp;

## Competency 10. **Commitment to Continuing Legal Education**

Continually seeks knowledge growth and professional development to stay updated with legal trends and regulations, reflecting the firm’s commitment to rich experience and unparalleled service.

### Lead Question

> **Tell me about a recent instance where you engaged in continuing legal education, including its relevance to your practice area. Follow-ups: How did you apply the knowledge gained? Was there a tangible improvement in your work? How do you ensure your ongoing education remains aligned with the firm's objectives and client needs?** 

_Look for specific examples of recent continued education, evidence of application, and alignment with firm goals and client services._


### Scenario

> Imagine our firm has recently taken on a case that involves recent changes in real estate law. How would you go about updating your knowledge to effectively handle this case?

_Look for a structured approach to learning, resource identification, timeline for education, and application to the case._

### Diving Deeper

* **Proactive Skill Enhancement**: "Describe a specific skill you’ve proactively enhanced through legal education and how it has contributed to your practice."
* **Education Relevance and Application**: "How do you choose legal educational activities and what criterion do you use to ensure their relevance to your litigation work?"
* **Knowledge Sharing**: "Can you provide an example of when you've shared legal insights or updates you've learned with colleagues to enhance team expertise?"
* **Anticipation of Legal Trends**: "Discuss how you stay ahead of legal trends and regulatory changes within the real estate and litigation field, and its importance for our firm."

### Evaluation

| Rating 1 | Rating 2 | Rating 3 | Rating 4 | Rating 5 |
| -------- | -------- | -------- | -------- | -------- |
|Generic response, no examples, or alignment.|Vague examples, minimal alignment with firm goals.|Adequate examples, some relevance and application.|Good examples, clear relevance, effective application.|Excellent examples, strong ties to practice, proactive alignment with firm vision.|


### Notes

&nbsp;





## Interview Evaluation for Real Estate Litigation Attorney

| Competency | N/A | 1 | 2 | 3 | 4 | 5 |
| --- | --- | --- | --- | --- | --- | --- |
| 1. In-depth Legal Expertise in Construction and Real Estate Litigation | | | | | | |
| 2. Analytical and Strategic Problem-Solving | | | | | | |
| 3. Exceptional Communication and Persuasion Skills | | | | | | |
| 4. Client-Centric Service Focus | | | | | | |
| 5. Effective Collaboration and Teamwork | | | | | | |
| 6. Adaptability in a Dynamic Environment | | | | | | |
| 7. Advocacy for Diversity and Inclusion | | | | | | |
| 8. Ethical Judgement and Professional Integrity | | | | | | |
| 9. Technological Proficiency | | | | | | |
| 10. Commitment to Continuing Legal Education | | | | | | |
| &nbsp; | | | | | | |
| &nbsp; | | | | | | |
| &nbsp; | | | | | | |
| &nbsp; | | | | | | |
| &nbsp; | **N/A** | **1** | **2** | **3** | **4** | **5** |
| **Totals** |   |   |   |   |   |   |




## Build Word docx

In [536]:
# Convert from markdown to docx
!pandoc -s {markdown_destination} -o {word_destination} --reference-doc="templates/custom-reference.docx" --toc=True --toc-depth=1 --shift-heading-level-by -1